In [0]:
# Read the csv file using spark df reader API
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
# Define schema
races_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                  StructField("year", IntegerType(), True),
                                  StructField("round", IntegerType(), True),
                                  StructField("circuitId", IntegerType(), True),
                                  StructField("name", StringType(), True),
                                  StructField("date", DateType(), True),
                                  StructField("time", StringType(), True),
                                  StructField("url", StringType(), True) 
])

In [0]:
# Read data in df 
races_df = spark.read \
.option("header", True) \
.schema(races_schema) \
.csv("/mnt/jdoformula1dl/raw/races.csv")

In [0]:
# Add ingestion date and race_timestamp column
from pyspark.sql.functions import current_timestamp, to_timestamp, concat, col, lit
races_with_timestamp_df = races_df.withColumn("ingestion_date", current_timestamp()) \
                                  .withColumn("race_timestamp", to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss'))

In [0]:
# Select relevant columns and rename for better clarity
races_selected_df = races_with_timestamp_df.select(col('raceId').alias('race_id'), col('year').alias('race_year'), 
                                                   col('round'), col('circuitId').alias('circuit_id'),col('name'), col('ingestion_date'), col('race_timestamp'))

In [0]:
# Write output as parquet file to the processed container
races_selected_df.write.mode('overwrite').partitionBy('race_year').parquet('/mnt/jdoformula1dl/processed/races')